# Loading Packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

## Progress Bar

In [2]:
%%capture
from tqdm import tqdm_notebook
tqdm_notebook().pandas()

## Pandas Options

In [3]:
pd.set_option('max_columns', None)
pd.set_option('display.max_rows', None)

# Loading Files

## 1 - Read Default Data

In [4]:
path = 'D:\\Desktop\\MLPGD_Capstone_Project\\resources\\aaa_sample_data.xlsx'

df_original = pd.read_excel(path)

* Data Glance

In [5]:
df_original.head(n=2)

,Column1,Individual Key,Household Key,Member Flag,City,State - Grouped,ZIP5,ZIP9,FSV CMSI Flag,FSV Credit Card Flag,FSV Deposit Program Flag,FSV Home Equity Flag,FSV ID Theft Flag,FSV Mortgage Flag,INS Client Flag,TRV Globalware Flag,Number of Children,Responded to Catalog,Race,Length Of Residence,Mail Responder,Home Owner,Income,Date Of Birth,Children,Education,Dwelling Type,Credit Ranges,Language,Gender,Active Expiration Date,Address Change Date,Bad Address Flag,Billing Code Description,Birth Date MMDDYYYY,Branch Name,Cancel Date,Cancel Reason,County,Do Not Direct Mail Solicit,Email Available,Email Status,ERS ENT Count Year 1,ERS ENT Count Year 2,ERS ENT Count Year 3,ERS Member Cost Year 1,ERS Member Cost Year 2,ERS Member Cost Year 3,Right_Gender,Right_Individual Key,Join AAA Date,Join Club Date,Member Key,Member Map Location,Member Number Associate ID,Member Phone Type,Member Status,Member Tenure Years,Member Type,Membership ID,Months from Join to Cancel,Opt-Out - Publication,Reason Joined,Reinstate Date,Renew Method,ZIP,Mosaic Household,Mosaic Global Household,kcl_B_IND_MosaicsGrouping,New Mover Flag,Occupation Code,Occupation Group,Right_Dwelling Type,Move Distance,Occupant Type,Breakdown Map Location,Breakdown City,Breakdown State,Basic Cost,Calculated Tow Miles,Call Canceled,Call Killed,Call Status Recv Date,Cash Call,Clearing Code Last Description,Dispatch Code1 Description,Dispatch Code2Description,DTL Prob1 Code Description,Fleet Indicator,Is Duplicate,Is NSR,Member Match Flag,Member Number and Associate ID,Motorcycle Indicator,Plus Cost,Plus Indicator Description,Premier Cost,Prob1 Code Description,Prob2 Code Description,SC Call Club Code Description,SC Date,Rec ID,SC STS RSN Code Description,SC Vehicle Manufacturer Name,SC Vehicle Model Name,SVC Facility Name,SVC Facility Type,Total Cost,Tow Destination Latitude,Tow Destination Longitude,Tow Destination Name,Was Duplicated,Was Towed To AAR Referral
0,0,100000030,104625900,Y,NEW HAVEN,CT,65110,651113490,N,N,N,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,Small or large multi-family w/apt number,NaN,NaN,Male,NaT,NaT,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,522115500,45007910,Y,WEST WARWICK,RI,28930,289338500,N,Y,N,N,N,N,N,N,One Child,NaN,Caucasion / White - English,150.0,Yes,Home Owner,"30-39,999",1922-02-05,Yes,Some College,Small or large multi-family w/apt number,600-649,English,NaN,2020-09-01,2005-01-28 12:41:09,F,New Member,1922-02-05,RI - WARWICK BRANCH,NaT,NaN,KENT,0.0,0.0,NaN,0.0,0.0,20.0,0.0,0.0,650.0,NaN,522115500.0,1970-09-01,1970-09-01,15000140.0,"{41.696,-71.5308}",153000.0,VoIP,ACTIVE,490.0,Primary,1530.0,NaN,Opt-In,NaN,2019-08-23,AUTO RENEW,28930.0,Senior Discounts,Low Income Elders,Golden Year Guardians,N,NaN,NaN,NaN,NaN,NaN,"{41,-71}",West Warwick,RI,325.0,8.0,N,N,2017-04-13 10:05:17,Y,CASH CALL ONLY C,Engine Overheat,NaN,Engine Overheat,N,0.0,0.0,10.0,153000.0,N,0.0,Basic Membership,0.0,Tow,NaN,AAA Northeast,2017-04-13,97073200.0,CASH CALL ONLY C,TOYOTA,CAMRY,ASTRO WRECKER SERVICE,independent repair,325.0,410.0,-710.0,Aar / Johnson's Auto Service,0.0,10.0


* General Info

In [6]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21344 entries, 0 to 21343
Columns: 113 entries, Column1 to Was Towed To AAR Referral
dtypes: datetime64[ns](9), float64(31), int64(5), object(68)
memory usage: 18.4+ MB


In [7]:
df_original.dtypes

Column1                                    int64
Individual Key                             int64
Household Key                              int64
Member Flag                               object
City                                      object
State - Grouped                           object
ZIP5                                       int64
ZIP9                                       int64
FSV CMSI Flag                             object
FSV Credit Card Flag                      object
FSV Deposit Program Flag                  object
FSV Home Equity Flag                      object
FSV ID Theft Flag                         object
FSV Mortgage Flag                         object
INS Client Flag                           object
TRV Globalware Flag                       object
Number of Children                        object
Responded to Catalog                      object
Race                                      object
Length Of Residence                      float64
Mail Responder      

* Reading Dictionaries

In [8]:
path_dict_member = 'D:\\Desktop\\MLPGD_Capstone_Project\\resources\\member_data_dict.xlsx'

dict_member = pd.read_excel(path_dict_member)

In [9]:
path_dict_rodaside = 'D:\\Desktop\\MLPGD_Capstone_Project\\resources\\roadside_data_dict.xlsx'

dict_roadside = pd.read_excel(path_dict_rodaside)

# Data processing (Exploratory Data Analysis - EDA)

* Creating Dictionary to keep track of each operation and each filtering per operation

In [10]:
removed_columns = {}

## 1 - Dropping Erroneous Column (excel did that on converting the csv original file)

    1.1 - Defining the key for this operation 

In [11]:
removed_columns['Erroneous Columns Removal'] = ['Column1'] 

    1.2 - Performing Filtering

In [12]:
df_original.drop(columns='Column1', inplace=True)

In [13]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21344 entries, 0 to 21343
Columns: 112 entries, Individual Key to Was Towed To AAR Referral
dtypes: datetime64[ns](9), float64(31), int64(4), object(68)
memory usage: 18.2+ MB


## 2 - Evaluating number of null values

In [14]:
nullval_ratio = (df_original.isnull().sum() * 100 / len(df_original)).sort_values(ascending=False) # percentage of null values per column

In [15]:
nullval_ratio; # Remove the ';' to print. BEWARE THE OUTPUT IS EXTENSIVE

    2.1 - Removing columns with more than 70% of null values

            2.1.1 - Maintaining Original Data file 

In [16]:
removed_null_data = df_original.copy()

            2.1.2 - Defining Threshold 

In [17]:
threshold = 70.0

            2.1.3 - Defining the key for this operation

In [18]:
key2 = str(threshold) + '% Threshold Column Removal'

removed_columns[key2] = []

            2.1.4 - Performing Filtering

In [19]:
for c in tqdm_notebook(df_original.columns, desc='Process Progress'):

    if nullval_ratio[c] > threshold:

        removed_null_data.drop(columns=c, inplace=True)

        removed_columns[key2].append(c)

print('Removed Columns after filtering process: ', removed_columns[key2])


Removed Columns after filtering process:  ['Responded to Catalog', 'Right_Gender', 'Reason Joined', 'Reinstate Date', 'Right_Dwelling Type', 'Move Distance', 'Occupant Type', 'Dispatch Code2Description', 'Prob2 Code Description', 'Tow Destination Name']


In [20]:
removed_null_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21344 entries, 0 to 21343
Columns: 102 entries, Individual Key to Was Towed To AAR Referral
dtypes: datetime64[ns](8), float64(30), int64(4), object(60)
memory usage: 16.6+ MB


## 3 - Subjective Evaluation

    3.1 - Rename FSV, INS, and TRV columns that are related to products purchase flags

In [21]:
matching_FSV = [s for s in list(removed_null_data.columns) if "FSV" in s]
matching_INS = [s for s in list(removed_null_data.columns) if "INS" in s]
matching_TRV = [s for s in list(removed_null_data.columns) if "TRV" in s]
final_matching = matching_FSV + matching_INS + matching_TRV
print(final_matching)

['FSV CMSI Flag', 'FSV Credit Card Flag', 'FSV Deposit Program Flag', 'FSV Home Equity Flag', 'FSV ID Theft Flag', 'FSV Mortgage Flag', 'INS Client Flag', 'TRV Globalware Flag']


In [22]:
counter = 1

for c in final_matching:

    removed_null_data.rename(columns={c : 'Purchased Product' + ' ' + str(counter)}, inplace=True)

    counter = counter + 1

In [23]:
removed_null_data.head(n=2)

,Individual Key,Household Key,Member Flag,City,State - Grouped,ZIP5,ZIP9,Purchased Product 1,Purchased Product 2,Purchased Product 3,Purchased Product 4,Purchased Product 5,Purchased Product 6,Purchased Product 7,Purchased Product 8,Number of Children,Race,Length Of Residence,Mail Responder,Home Owner,Income,Date Of Birth,Children,Education,Dwelling Type,Credit Ranges,Language,Gender,Active Expiration Date,Address Change Date,Bad Address Flag,Billing Code Description,Birth Date MMDDYYYY,Branch Name,Cancel Date,Cancel Reason,County,Do Not Direct Mail Solicit,Email Available,Email Status,ERS ENT Count Year 1,ERS ENT Count Year 2,ERS ENT Count Year 3,ERS Member Cost Year 1,ERS Member Cost Year 2,ERS Member Cost Year 3,Right_Individual Key,Join AAA Date,Join Club Date,Member Key,Member Map Location,Member Number Associate ID,Member Phone Type,Member Status,Member Tenure Years,Member Type,Membership ID,Months from Join to Cancel,Opt-Out - Publication,Renew Method,ZIP,Mosaic Household,Mosaic Global Household,kcl_B_IND_MosaicsGrouping,New Mover Flag,Occupation Code,Occupation Group,Breakdown Map Location,Breakdown City,Breakdown State,Basic Cost,Calculated Tow Miles,Call Canceled,Call Killed,Call Status Recv Date,Cash Call,Clearing Code Last Description,Dispatch Code1 Description,DTL Prob1 Code Description,Fleet Indicator,Is Duplicate,Is NSR,Member Match Flag,Member Number and Associate ID,Motorcycle Indicator,Plus Cost,Plus Indicator Description,Premier Cost,Prob1 Code Description,SC Call Club Code Description,SC Date,Rec ID,SC STS RSN Code Description,SC Vehicle Manufacturer Name,SC Vehicle Model Name,SVC Facility Name,SVC Facility Type,Total Cost,Tow Destination Latitude,Tow Destination Longitude,Was Duplicated,Was Towed To AAR Referral
0,100000030,104625900,Y,NEW HAVEN,CT,65110,651113490,N,N,N,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,Small or large multi-family w/apt number,NaN,NaN,Male,NaT,NaT,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,522115500,45007910,Y,WEST WARWICK,RI,28930,289338500,N,Y,N,N,N,N,N,N,One Child,Caucasion / White - English,150.0,Yes,Home Owner,"30-39,999",1922-02-05,Yes,Some College,Small or large multi-family w/apt number,600-649,English,NaN,2020-09-01,2005-01-28 12:41:09,F,New Member,1922-02-05,RI - WARWICK BRANCH,NaT,NaN,KENT,0.0,0.0,NaN,0.0,0.0,20.0,0.0,0.0,650.0,522115500.0,1970-09-01,1970-09-01,15000140.0,"{41.696,-71.5308}",153000.0,VoIP,ACTIVE,490.0,Primary,1530.0,NaN,Opt-In,AUTO RENEW,28930.0,Senior Discounts,Low Income Elders,Golden Year Guardians,N,NaN,NaN,"{41,-71}",West Warwick,RI,325.0,8.0,N,N,2017-04-13 10:05:17,Y,CASH CALL ONLY C,Engine Overheat,Engine Overheat,N,0.0,0.0,10.0,153000.0,N,0.0,Basic Membership,0.0,Tow,AAA Northeast,2017-04-13,97073200.0,CASH CALL ONLY C,TOYOTA,CAMRY,ASTRO WRECKER SERVICE,independent repair,325.0,410.0,-710.0,0.0,10.0


    3.2 -  Removing columns that are not allowed or not ethical to work


        3.2.1 - Creating Dictionary Key

In [24]:
key3 = 'Unethical Columns'

removed_columns[key3] = ['Race', 'Language', 'Gender'] # is gender an unethical information for market segmentation?

        3.2.2 - Performing Filtering

In [25]:
for c in tqdm_notebook(removed_columns[key3], desc='Process Progress'):

    removed_null_data.drop(columns=c, inplace=True)

print('Removed Columns after filtering process: ', removed_columns[key3])


Removed Columns after filtering process:  ['Race', 'Language', 'Gender']


In [26]:
removed_null_data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21344 entries, 0 to 21343
Data columns (total 99 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Individual Key                  21344 non-null  int64         
 1   Household Key                   21344 non-null  int64         
 2   Member Flag                     21344 non-null  object        
 3   City                            21344 non-null  object        
 4   State - Grouped                 21344 non-null  object        
 5   ZIP5                            21344 non-null  int64         
 6   ZIP9                            21344 non-null  int64         
 7   Purchased Product 1             21344 non-null  object        
 8   Purchased Product 2             21344 non-null  object        
 9   Purchased Product 3             21344 non-null  object        
 10  Purchased Product 4             21344 non-null  object        
 11  Pu

    3.3 - Removing Columns that represent the same information in a different way (information redundancy)

        3.3.1 - Creating Dictionary Key

In [27]:
key4 = 'Redundancy Information Columns'

removed_columns[key4] = ['Individual Key', 'State - Grouped', 'ZIP5', 'ZIP9', 'Children', 'Birth Date MMDDYYYY', 'Cancel Date', 'County', 'Do Not Direct Mail Solicit', 'Right_Individual Key', 'Member Key', 'Member Number Associate ID', 'Membership ID', 'Reinstate Date', 'ZIP', 'Mosaic Household', 'kcl_B_IND_MosaicsGrouping', 'Occupation Code', 'Breakdown State', 'Call Killed', 'Clearing Code Last Description', 'Dispatch Code1 Description','DTL Prob1 Code Description', 'Is Duplicate', 'Member Match Flag', 'Member Number and Associate ID', 'SC Date', 'Rec ID', 'SC STS RSN Code Description', 'SC Vehicle Model Name', 'SVC Facility Name', 'SVC Facility Type', 'Tow Destination Latitude', 'Tow Destination Longitude'] # is the City variable relevant once we already have lat/long location? Address Change Date? Bad Adress Flag? Reinstate Date (y)? Call Status Recv Date? 

        3.3.2 - Performing Filtering

In [28]:
for c in tqdm_notebook(removed_columns[key4], desc='Process Progress'):

    if c in removed_null_data.columns: # since we have choosen a lot of columns, we have to ensure that all of them are still in the current dataset status

        removed_null_data.drop(columns=c, inplace=True)

print('Removed Columns after filtering process: ', removed_columns[key4])


Removed Columns after filtering process:  ['Individual Key', 'State - Grouped', 'ZIP5', 'ZIP9', 'Children', 'Birth Date MMDDYYYY', 'Cancel Date', 'County', 'Do Not Direct Mail Solicit', 'Right_Individual Key', 'Member Key', 'Member Number Associate ID', 'Membership ID', 'Reinstate Date', 'ZIP', 'Mosaic Household', 'kcl_B_IND_MosaicsGrouping', 'Occupation Code', 'Breakdown State', 'Call Killed', 'Clearing Code Last Description', 'Dispatch Code1 Description', 'DTL Prob1 Code Description', 'Is Duplicate', 'Member Match Flag', 'Member Number and Associate ID', 'SC Date', 'Rec ID', 'SC STS RSN Code Description', 'SC Vehicle Model Name', 'SVC Facility Name', 'SVC Facility Type', 'Tow Destination Latitude', 'Tow Destination Longitude']


In [29]:
removed_null_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21344 entries, 0 to 21343
Data columns (total 66 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Household Key                  21344 non-null  int64         
 1   Member Flag                    21344 non-null  object        
 2   City                           21344 non-null  object        
 3   Purchased Product 1            21344 non-null  object        
 4   Purchased Product 2            21344 non-null  object        
 5   Purchased Product 3            21344 non-null  object        
 6   Purchased Product 4            21344 non-null  object        
 7   Purchased Product 5            21344 non-null  object        
 8   Purchased Product 6            21344 non-null  object        
 9   Purchased Product 7            21344 non-null  object        
 10  Purchased Product 8            21344 non-null  object        
 11  Number of Child

    3.4 - Checking Columns with low percentage of variantional information (counting null values)

        3.4.1 - Creating Dictionay Key 

In [30]:
key5 = 'Low Variational Information Columns'

removed_columns[key5] = []

        3.4.2 - Performing Filtering

In [31]:
for c in tqdm_notebook(removed_null_data.columns, desc='Process Progress'):

    if removed_null_data.nunique()[c] == 1:

        removed_null_data.drop(columns=c, inplace=True)

        removed_columns[key5].append(c)
        
print('Removed Columns after filtering process: ', removed_columns[key5])


Removed Columns after filtering process:  ['Member Flag']


In [32]:
removed_null_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21344 entries, 0 to 21343
Data columns (total 65 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Household Key                  21344 non-null  int64         
 1   City                           21344 non-null  object        
 2   Purchased Product 1            21344 non-null  object        
 3   Purchased Product 2            21344 non-null  object        
 4   Purchased Product 3            21344 non-null  object        
 5   Purchased Product 4            21344 non-null  object        
 6   Purchased Product 5            21344 non-null  object        
 7   Purchased Product 6            21344 non-null  object        
 8   Purchased Product 7            21344 non-null  object        
 9   Purchased Product 8            21344 non-null  object        
 10  Number of Children             12010 non-null  object        
 11  Length Of Resid

    3.5 - Removing Columns with Unnecessary Information (extremely subjective)

        3.5.1 - Creating Dictionary Key

In [33]:
key6 = 'Unnecessary Columns'

removed_columns[key6] = ['Address Change Date', 'Bad Address Flag', 'Billing Code Description', 'Join Club Date', 'Member Phone Type', 'Mosaic Global Household', 'New Mover Flag', 'Call Canceled', 'Is NSR', 'Plus Indicator Description', 'Calculated Tow Miles', 'Was Duplicated', 'Was Towed To AAR Referral', 'Branch Name']

        3.5.2 - Performing Filtering

In [34]:
for c in tqdm_notebook(removed_columns[key6], desc='Process Progress'):

    removed_null_data.drop(columns=c, inplace=True)

print('Removed Columns after filtering process: ', removed_columns[key6])


Removed Columns after filtering process:  ['Address Change Date', 'Bad Address Flag', 'Billing Code Description', 'Join Club Date', 'Member Phone Type', 'Mosaic Global Household', 'New Mover Flag', 'Call Canceled', 'Is NSR', 'Plus Indicator Description', 'Calculated Tow Miles', 'Was Duplicated', 'Was Towed To AAR Referral', 'Branch Name']


In [35]:
removed_null_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21344 entries, 0 to 21343
Data columns (total 51 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Household Key                  21344 non-null  int64         
 1   City                           21344 non-null  object        
 2   Purchased Product 1            21344 non-null  object        
 3   Purchased Product 2            21344 non-null  object        
 4   Purchased Product 3            21344 non-null  object        
 5   Purchased Product 4            21344 non-null  object        
 6   Purchased Product 5            21344 non-null  object        
 7   Purchased Product 6            21344 non-null  object        
 8   Purchased Product 7            21344 non-null  object        
 9   Purchased Product 8            21344 non-null  object        
 10  Number of Children             12010 non-null  object        
 11  Length Of Resid

## 4 - Summarize Information and Fill Nulls in the given columns 

    Renaming Pivot Variable and saving previous state of the data

In [36]:
filtered_data = removed_null_data.copy()

    4.1 - Number of Purchased Products

        4.1.1 - Creating Dictionary Key

In [37]:
key7 = 'Merging total number of purchased products by each row'

removed_columns[key7] = ['Purchased Product 1',	'Purchased Product 2',	'Purchased Product 3',	'Purchased Product 4',	'Purchased Product 5',	'Purchased Product 6',	'Purchased Product 7',	'Purchased Product 8']

        4.1.2 - Performing Summary

            4.1.2.1 - Converting String Values in Numerical Values and filling Null values as 0 purchased products for empty different Household Keys

In [38]:
for idx in tqdm_notebook(range(len(filtered_data)), desc='Process Progress'): # For each row
    for c in removed_columns[key7]: # In each Purchased Product Column

        if (filtered_data[c].values[idx] == 'N') or (filtered_data[c].values[idx] == 'Null') or (filtered_data[c].values[idx] == np.nan):

            filtered_data[c].values[idx] = 0

        else:

            filtered_data[c].values[idx] = 1

            4.1.2.2 - Converting Columns to Iterger Type

In [39]:
for c in tqdm_notebook(removed_columns[key7], desc='Process Progress'):

    filtered_data[c] = filtered_data[c].astype(int)

        4.1.2.3 - Summarize all Purchased Columns

In [40]:
new_column = [] # creating list for the new column to be appended

column_name = 'Total Number of Purchased Products' # naming the new column

for idx in tqdm_notebook(range(len(filtered_data)), desc='Process Progress'): # For each row
    total_amount = 0 # total amount of products bought in each row

    for c in removed_columns[key7]: # In each Purchased Product Column

        if filtered_data[c][idx] == 1:

            total_amount = total_amount + 1

    new_column.append(total_amount)

        4.1.2.4 - Drop Separate Purchased Data and Concatenate Total Purchased Data

In [41]:
for c in tqdm_notebook(removed_columns[key7], desc='Process Progress'):

    filtered_data.drop(columns=c, inplace=True)

filtered_data[column_name] = new_column
print('Removed Columns after filtering process: ', removed_columns[key7])


Removed Columns after filtering process:  ['Purchased Product 1', 'Purchased Product 2', 'Purchased Product 3', 'Purchased Product 4', 'Purchased Product 5', 'Purchased Product 6', 'Purchased Product 7', 'Purchased Product 8']


In [42]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21344 entries, 0 to 21343
Data columns (total 44 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Household Key                       21344 non-null  int64         
 1   City                                21344 non-null  object        
 2   Number of Children                  12010 non-null  object        
 3   Length Of Residence                 12010 non-null  float64       
 4   Mail Responder                      12010 non-null  object        
 5   Home Owner                          11629 non-null  object        
 6   Income                              11943 non-null  object        
 7   Date Of Birth                       10191 non-null  datetime64[ns]
 8   Education                           12010 non-null  object        
 9   Dwelling Type                       15891 non-null  object        
 10  Credit Ranges         

    4.2 - Correcting Number of Children column

        4.2.1 - Checking if for any HouseHold Key there is a different information for children numbering

In [43]:
Counter(df_original.groupby('Individual Key')['Number of Children'].nunique().values) # as the sheet is sorted for Individual Keys, since the beginning, we need to check if, for each Individual Key there is only 1 or 0 entries for the Number of Children

Counter({0: 6076, 1: 5827})

            With this information, we can ensure that there is no Individual Key, therefore no Household Key since it is a collection of the previous, with an erroneous entry for the Number of Children column

        4.2.2 - Converting String Values in Numerical Values and filling Null values as 0 children for empty different Household Keys

In [44]:
mask_n_children = filtered_data['Number of Children'].isnull() # creating mask of null values in Number of Children column

In [45]:
for idx in tqdm_notebook(range(len(filtered_data)), desc='Process Progress'): # For each row

    if mask_n_children[idx]: # if the mask is true, that means that the correponded value for that index is Null

        filtered_data['Number of Children'].values[idx] = 0    
    
    else:

        entry = filtered_data['Number of Children'].values[idx].split()[0]

        if (entry == 'No'):

            filtered_data['Number of Children'].values[idx] = 0

        elif (entry == 'One'):

            filtered_data['Number of Children'].values[idx] = 1

        elif (entry == 'Two'):

            filtered_data['Number of Children'].values[idx] = 2

        elif (entry == 'Three'):

            filtered_data['Number of Children'].values[idx] = 3

        elif (entry == 'Four'):

            filtered_data['Number of Children'].values[idx] = 4

        elif (entry == 'Five'):

            filtered_data['Number of Children'].values[idx] = 5

        elif (entry == 'Six'):

            filtered_data['Number of Children'].values[idx] = 6

        4.2.2 - Converting Column to Iterger Type


In [46]:
filtered_data['Number of Children'] = filtered_data['Number of Children'].astype(int)

In [47]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21344 entries, 0 to 21343
Data columns (total 44 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Household Key                       21344 non-null  int64         
 1   City                                21344 non-null  object        
 2   Number of Children                  21344 non-null  int32         
 3   Length Of Residence                 12010 non-null  float64       
 4   Mail Responder                      12010 non-null  object        
 5   Home Owner                          11629 non-null  object        
 6   Income                              11943 non-null  object        
 7   Date Of Birth                       10191 non-null  datetime64[ns]
 8   Education                           12010 non-null  object        
 9   Dwelling Type                       15891 non-null  object        
 10  Credit Ranges         

    4.3 - Length of Residence

    The main approach we are going to apply here is to get the mean value of the household key and apply it on the same number. After that, if it still exists any null value on the given column, we will use the mean or the mode of the column City to fill it.

        4.3.1 - Replace random default value apllied by Pandas on Null values for the Length Of Residence column. For some reason it was choosen a smaller number as possible for a float64 number

In [48]:
filtered_data['Length Of Residence'].unique()  # as we can see, by some reason, the number -9.223372e+18 was choosen to be a flag of a Null value for this column, we need to replace it by 0 or np.nan

array([ nan, 150., 130.,  50., 110.,  70.,  30., 100.,   0.,  80., 140.,
        60., 120.,  20., 230.,  10.,  40.,  90., 390., 340., 190., 380.,
       180., 250., 220., 470., 200., 440., 210., 170., 330., 160., 310.,
       460., 320., 260., 280., 290., 270., 240., 360., 350., 490., 430.,
       450.])

In [49]:
filtered_data['Length Of Residence'] = filtered_data['Length Of Residence'].replace(filtered_data['Length Of Residence'].unique()[0], np.nan) # replacing random numerical Null values chose by Pandas for numpy NaN 

In [50]:
filtered_data['Length Of Residence'].unique() # now we have out correct output for Null values for the Length Of Residence column

array([ nan, 150., 130.,  50., 110.,  70.,  30., 100.,   0.,  80., 140.,
        60., 120.,  20., 230.,  10.,  40.,  90., 390., 340., 190., 380.,
       180., 250., 220., 470., 200., 440., 210., 170., 330., 160., 310.,
       460., 320., 260., 280., 290., 270., 240., 360., 350., 490., 430.,
       450.])

        4.3.2 - Checking if there are Household Keys with at least one value for the Length Of Residence column and apply it to the rest of them

In [51]:
filtered_data.groupby('Household Key')['Length Of Residence'].unique(); # As we can see, for the same Household Key level we have different values of Length Of Residence. This happens because our main sheet data was sorted on a Individual Key level, that is not our interest here. # OBS: REMOVE THE ';' AT THE END OF THE COMMAND TO PRINT THE OUTPUT! BE ADVISED, THE OUTPUT IS EXTENSE!

In [52]:
for house_id in tqdm_notebook(filtered_data.groupby('Household Key')['Length Of Residence'].unique().keys(), desc='Process Progress'): # for every single unique Household Key value

    if len(filtered_data.groupby('Household Key')['Length Of Residence'].unique()[house_id]) == 1: # if there is only one associated value for Length Of Residence, independet of the value, we can infer nothing for now, so, we pass
        pass
    
    else:

        tmp = filtered_data.loc[filtered_data['Household Key'] == house_id]['Length Of Residence'].values # storing array on a temporary file

        if np.isnan(tmp).any(): # check if the associated values for the Length Of Residence for a given Household Key has at least one Null value

            nan_positions = np.where(np.isnan(tmp))[0].tolist() # grabing the positions where the Null values exist

            if len(nan_positions) == len(tmp): # checking if all the values for the given Household Key are Null (in this case we will treat it in another way, so, we pass)
                pass

            else:

                replacement_value =  tmp[~np.isnan(tmp)].mean() # calculating the mean of all non Null values

                tmp[np.isnan(tmp)] = replacement_value # index of Null values on the array

                idxs = filtered_data.loc[filtered_data['Household Key'] == house_id, 'Length Of Residence'].index[nan_positions] # index of Null values on the dataframe

                for i in idxs: # for each dataframe index
                
                    filtered_data.at[i, 'Length Of Residence'] = replacement_value # replace de Null value by the Household Key mean

        4.3.3 - Now we focus on predicting the rest of the Null values, but this time through groups of cities

In [53]:
filtered_data.groupby('City')['Length Of Residence'].unique(); # As we can see, for the same City we have different values of Length Of Residence. We want to take advantage of this bigger scale to replace more Null values # OBS: REMOVE THE ';' AT THE END OF THE COMMAND TO PRINT THE OUTPUT! BE ADVISED, THE OUTPUT IS EXTENSE!

In [54]:
for city in tqdm_notebook(filtered_data.groupby('City')['Length Of Residence'].unique().keys(), desc='Process Progress'): # for every single unique Household Key value

    if len(filtered_data.groupby('City')['Length Of Residence'].unique()[city]) == 1: # if there is only one associated value for Length Of Residence, we can infer nothing for now. So, we pass and let this for the state mean 
        pass
    
    else:

        tmp = filtered_data.loc[filtered_data['City'] == city]['Length Of Residence'].values # storing array on a temporary file

        if np.isnan(tmp).any(): # check if the associated values for the Length Of Residence for a given City has at least one Null value

            nan_positions = np.where(np.isnan(tmp))[0].tolist() # grabing the positions where the Null values exist

            if len(nan_positions) == len(tmp): # checking if all the values for the given City are Null (in this case we will treat it in another way, so, we pass)
                pass

            else:

                replacement_value =  tmp[~np.isnan(tmp)].mean() # calculating the mean of all non Null values

                tmp[np.isnan(tmp)] = replacement_value # index of Null values on the array

                idxs = filtered_data.loc[filtered_data['City'] == city, 'Length Of Residence'].index[nan_positions] # index of Null values on the dataframe

                for i in idxs: # for each dataframe index
                
                    filtered_data.at[i, 'Length Of Residence'] = replacement_value # replace de Null value by the City mean

            4.3.4 - Now we focus on predicting the rest of the Null values on the biggest and final scale as possible, the states.

            Note: You may noticed that we have excluded the State - Gouped column, but, since we havent eliminated any row yet we can still use the previous indexes of this unusable column 

In [55]:
ct_idxs = df_original.index[df_original['State - Grouped'] == 'CT'].tolist() # grab index of all rows that belong to the CT state
ri_idxs = df_original.index[df_original['State - Grouped'] == 'RI'].tolist() # grab index of all rows that belong to the RI state

In [56]:
ct_mean = filtered_data['Length Of Residence'][ct_idxs].mean()
ri_mean = filtered_data['Length Of Residence'][ri_idxs].mean()
print('CT mean value considering all Null values so far filled: ', ct_mean, '\n', 'RI mean value considering all Null values so far filled: ', ri_mean)

CT mean value considering all Null values so far filled:  73.08636311810243 
 RI mean value considering all Null values so far filled:  112.58786399700816


In [57]:
for idx in tqdm_notebook(filtered_data.index, desc='Process Progress'): # for every single unique Household Key value

    if filtered_data['Length Of Residence'].isnull()[idx]: # if for the given index of the Length Of Residence column the value is Null
        if idx in ct_idxs: # check if this index belongs to the CT state

            filtered_data.at[idx, 'Length Of Residence'] = ct_mean # replace the Null value by the CT state mean

        else:

            filtered_data.at[idx, 'Length Of Residence'] = ri_mean # replace the Null value by the RI state mean

In [58]:
filtered_data['Length Of Residence'].isnull().values.any() # check if Length Of Residence column still has any Null value left

False

In [59]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21344 entries, 0 to 21343
Data columns (total 44 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Household Key                       21344 non-null  int64         
 1   City                                21344 non-null  object        
 2   Number of Children                  21344 non-null  int32         
 3   Length Of Residence                 21344 non-null  float64       
 4   Mail Responder                      12010 non-null  object        
 5   Home Owner                          11629 non-null  object        
 6   Income                              11943 non-null  object        
 7   Date Of Birth                       10191 non-null  datetime64[ns]
 8   Education                           12010 non-null  object        
 9   Dwelling Type                       15891 non-null  object        
 10  Credit Ranges         

    4.4 - Summarize E-mail and Mail Information

        4.4.1 - Creating List of Columns to be summarized

In [60]:
mail_var = ['Mail Responder', 'Email Available', 'Email Status', 'Opt-Out - Publication']

        4.4.2 - Checking Existance of Null Values on this columns

In [61]:
for col in mail_var:

    print('Flag if column ', col, ' has null values. ', filtered_data[col].isnull().values.any(), ' and its percentage ', nullval_ratio[col]) # check if the columns have any Null values

Flag if column  Mail Responder  has null values.  True  and its percentage  43.731259370314845
Flag if column  Email Available  has null values.  True  and its percentage  0.0046851574212893555
Flag if column  Email Status  has null values.  True  and its percentage  48.83339580209895
Flag if column  Opt-Out - Publication  has null values.  True  and its percentage  0.0046851574212893555


        As we can see, all the binary categorical columns have Null values. However, the columns related to Email Available and Opt-Out - Publication are almost full filled and, according to the spreadsheet dictionary, those two are AAA direct information and not third party information. So, we will focus on them as most reliable data and use the Member Status as well to "predict" a summary of all these information

In [83]:
filtered_data.groupby('Household Key')['Email Status'].unique();

array([nan, 'Unsubscribed', 'Active'], dtype=object)

        4.4.3 - Give the name of the new column to be added on the dataframe

In [89]:
new_mail_key = 'Email Correspondent' # new column name

filtered_data[new_mail_key] = "" # appending new column to the dataframe

        4.4.4 - To summarize all these information we need to ensure that for the same Household Key we have a concordance between all the variables. For that we will use the Member Status variable and Email Available as pivot information

In [86]:
for house_id in tqdm_notebook(filtered_data['Household Key'].unique().tolist(), desc='Process Progress'): # for every single unique Household Key value

    if len(Counter(filtered_data.groupby('Household Key')['Member Status'].unique()[house_id].tolist())) != 1: # if there is more than one Member Status type for the given Household Key

        if 'ACTIVE' in filtered_data.groupby('Household Key')['Member Status'].unique()[house_id].tolist(): # if at least one individual member is active on the given HouseHold Key

            filtered_data.loc[(filtered_data['Household Key'] == house_id), 'Member Status'] = 'ACTIVE' # replace them all for active

        elif 'PENDING' in filtered_data.groupby('Household Key')['Member Status'].unique()[house_id].tolist(): # if at least one individual member is pending on the given HouseHold Key and there is none active

            filtered_data.loc[(filtered_data['Household Key'] == house_id), 'Member Status'] = 'PENDING' # replace them all for pending 
    
    if len(Counter(filtered_data.groupby('Household Key')['Email Available'].unique()[house_id].tolist())) != 1: # if there is more than one Email Available type for the given Household Key

        if 10 in filtered_data.groupby('Household Key')['Email Available'].unique()[house_id].tolist(): # if at least one individual member has its email available on the given HouseHold Key

            filtered_data.loc[(filtered_data['Household Key'] == house_id), 'Email Available'] = 10 # replace them all by 10 (active)

    if len(Counter(filtered_data.groupby('Household Key')['Opt-Out - Publication'].unique()[house_id].tolist())) != 1: # if there is more than one Opt-Out - Publication type for the given Household Key   

        if 'Opt-In' in filtered_data.groupby('Household Key')['Opt-Out - Publication'].unique()[house_id].tolist(): # if at least one individual member has optional for an email publication on the given HouseHold Key

            filtered_data.loc[(filtered_data['Household Key'] == house_id), 'Opt-Out - Publication'] = 'Opt-In' # replace them all by Opt-In (active)


        4.4.5 - Filling the new column

In [93]:
for house_id in tqdm_notebook(filtered_data['Household Key'].unique().tolist(), desc='Process Progress'): # for every single unique Household Key value

    if ('ACTIVE' in filtered_data[filtered_data['Household Key'] == house_id]['Member Status'].tolist()) and (10 in filtered_data[filtered_data['Household Key'] == house_id]['Email Available'].tolist()) and ('Opt-In' in filtered_data[filtered_data['Household Key'] == house_id]['Opt-Out - Publication'].tolist()): # if all the conditions are satisfied

        filtered_data.loc[(filtered_data['Household Key'] == house_id), 'Email Correspondent'] = 1 # the household is an e-mail correspondent

    else:

        filtered_data.loc[(filtered_data['Household Key'] == house_id), 'Email Correspondent'] = 0 # the household is not an e-mail correspondent


filtered_data['Email Correspondent'] = filtered_data['Email Correspondent'].astype(int) # changing to integer data type

        4.4.6 -  Dropping Email information Columns

In [94]:
key8 = 'Merging Availability of Email Contact'

removed_columns[key8] = mail_var

In [95]:
for c in mail_var:

    filtered_data.drop(columns=c, inplace=True)

print('Removed Columns after filtering process: ', removed_columns[key8])

Removed Columns after filtering process:  ['Mail Responder', 'Email Available', 'Email Status', 'Opt-Out - Publication']


In [100]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21344 entries, 0 to 21343
Data columns (total 41 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Household Key                       21344 non-null  int64         
 1   City                                21344 non-null  object        
 2   Number of Children                  21344 non-null  int32         
 3   Length Of Residence                 21344 non-null  float64       
 4   Home Owner                          11629 non-null  object        
 5   Income                              11943 non-null  object        
 6   Date Of Birth                       10191 non-null  datetime64[ns]
 7   Education                           12010 non-null  object        
 8   Dwelling Type                       15891 non-null  object        
 9   Credit Ranges                       12010 non-null  object        
 10  Active Expiration Date

In [101]:
path_out = '..\\resources\\checkpoint.csv'
filtered_data.to_csv(path_or_buf=path_out, index=False)

        We still need to keep an eye on the variables that only have 2 unique values, but it will be more a subjective evaluation 

* Check correlation Matrix

#correlation matrix
corrmat = remove_null_data.corr()
f, ax = plt.subplots(figsize=(15, 10))
sns.heatmap(corrmat, square=True); # considering 70% o correlation as minimum to show

* Observations

  1- There is an "island" between Rec ID and Tow destination Longitude this will be evaluate latter, but looks promissing
  2- Basic Cost has a high correlation with the variables within this "island" 
  3- Individual Key and Right_Individual Key are pratically the same variable
  4- ZIP5 and ZIP9 are pratically the same variable
  5- Months from join to Cancel has no correlation at so ever with the Premier Cost variable
  

    By the correlation matrix of the entire dataset (minus the columns with more than 70% of null values), we can see that there are still some columns with no correlation, or total correlation with all the variables, and some trouble columns. Therefore, the best thing to do is to drop them.

trouble_columns= ['Is Duplicate', 'Member Match Flag']

columns_will_not_use = ['Individual Key', 'Member Flag', 'Right_Individual Key']